<a href="https://colab.research.google.com/github/Imtiazul-Islam/upsclae-video/blob/main/real_upscale_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cell 1: Environment Setup & Library Patching**

This cell installs the necessary tools and applies the "Hard Fix" to the basicsr library so it works with modern Python 3.12/Torch versions.

In [ ]:
# @title
# ==========================================================
# STEP 1: INSTALLATION & ENVIRONMENT PATCHING
# ==========================================================
import torch
import os

# Check for GPU (Essential for Real-ESRGAN)
assert torch.cuda.is_available(), "GPU not detected. Please change runtime to GPU."

print("Cloning Real-ESRGAN Repository...")
!rm -rf Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN

print("Installing Dependencies (This may take a minute)...")
# Installing facexlib and gfpgan for face enhancement support
!pip install -q facexlib gfpgan ffmpeg-python basicsr
!pip install -q -r requirements.txt
!python setup.py develop

print("Applying torchvision compatibility patch...")
# FIX: The 'basicsr' library has a broken import in modern Torchvision.
# This 'sed' command replaces 'functional_tensor' with 'functional' in the source code.
!sed -i 's/torchvision.transforms.functional_tensor/torchvision.transforms.functional/g' /usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py

try:
    import basicsr
    import cv2
    print("✅ SUCCESS: Environment is ready and basicsr is patched!")
except Exception as e:
    print(f"❌ ERROR: Setup failed. Details: {e}")

**Cell 2: Parameters & Configuration**

This cell allows you to define your input file and choose the high-quality 6B model for the best results on your anime-style cat videos.

In [ ]:
# ==========================================================
# STEP 2: OPTIMIZED RESOLUTION & MODEL SELECTION
# ==========================================================
import cv2
import os

video_path = "/content/fox-playing-mbl.mp4" #@param {type:"string"}
output_dir = "/content/" #@param {type:"string"}
resolution = "2x Original" #@param ["FHD (1920 x 1080)", "2k (2560 x 1440)", "4k (3840 x 2160)", "8k (7680 x 4320)", "2x Original", "3x Original", "4x Original"]

# SPEED TIP: 'realesr-animevideov3' is 10x faster for animation.
# For your iQOO Z10x screen, RealESRGAN_x4plus is high quality but heavy.
model_name = "realesr-animevideov3" #@param ["RealESRGAN_x4plus", "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]

if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video not found: {video_path}")

cap = cv2.VideoCapture(video_path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) # Captured for better FFmpeg matching
cap.release()

res_map = {
    "FHD (1920 x 1080)": (1920, 1080),
    "2k (2560 x 1440)": (2560, 1440),
    "4k (3840 x 2160)": (3840, 2160),
    "8k (7680 x 4320)": (7680, 4320),
    "2x Original": (w*2, h*2),
    "3x Original": (w*3, h*3),
    "4x Original": (w*4, h*4)
}

target_w, target_h = res_map[resolution]

# Aspect Ratio Correction for Vertical Videos (like your 704x1280)
if (w < h and target_w > target_h):
    target_w, target_h = target_h, target_w

scale_factor = target_w / w

print(f"✅ Source: {w}x{h} @ {fps} FPS")
print(f"🚀 Target: {target_w}x{target_h} ({scale_factor:.2f}x scale)")

**Cell 3: Execution & Final Encoding**

This runs the AI processing and uses FFmpeg with high-bitrate settings to ensure no detail is lost during the final save.

In [ ]:
# @title
# ==========================================================
# STEP 3: HIGH-SPEED EXECUTION (CORRECTED FLAGS)
# ==========================================================
import subprocess

base_name = os.path.splitext(os.path.basename(video_path))[0]
raw_output = os.path.join(output_dir, f"{base_name}_out.mp4")
final_output = os.path.join(output_dir, f"{base_name}_Upscaled_{resolution.split(' ')[0]}.mp4")

# --tile 256: Smaller tiles are actually FASTER on Colab because they fit
# into the GPU's L2 cache better and avoid the "Slow IO" bottleneck.
tile_setting = "256"

# Note: We removed --half. The script uses FP16 by default.
# If it still fails, we add --fp32 (but that will be slower).
upscale_cmd = (
    f"python inference_realesrgan_video.py "
    f"-n {model_name} "
    f"-i '{video_path}' "
    f"-o '{output_dir}' "
    f"-s {scale_factor} "
    f"-t {tile_setting}"
)

print(f"⚡ Running Optimized Inference...")
print(f"Executing: {upscale_cmd}")
!{upscale_cmd}

if os.path.exists(raw_output):
    print("🎬 AI processing complete. Encoding The Final Video...")
    # Using -c:v libx264 with 'fast' preset for a balance of speed and quality
    # Included -y to overwrite and -pix_fmt for Android compatibility
    ffmpeg_cmd = (
        f"ffmpeg -loglevel error -y -i '{raw_output}' "
        f"-c:v libx264 -preset fast -crf 20 -pix_fmt yuv420p '{final_output}'"
    )
    subprocess.run(ffmpeg_cmd, shell=True)

    print(f"✨ ALL DONE!")
    print(f"📍 Final File: {final_output}")
    if os.path.exists(raw_output): os.remove(raw_output)
else:
    print("❌ ERROR: Output file not found. Try reducing the scale or checking if the disk is full.")